# DEV DR-Tulu

## Evolucion test

In [1]:
import json
import os
import re
import asyncio
from typing import List, Dict, Any, Optional

# Cargar variables de entorno desde .env si existe
try:
    from dotenv import load_dotenv
    load_dotenv()  # Carga .env automáticamente
except ImportError:
    # Si no está instalado dotenv, simplemente no carga el .env
    pass

# Solo necesitamos OpenAI (o puedes usar Anthropic, etc.)
try:
    from openai import AsyncOpenAI, AsyncAzureOpenAI
except ImportError:
    print("Error: Necesitas instalar openai: pip install openai")
    exit(1)

In [2]:
# from drtulu_rubric_evolve_simple import evaluar_respuesta_completa, evaluar_rubrica, generar_rubricas_adaptativas, actualizar_ground_truth

In [ ]:
# ============================================================================
# CONFIGURACIÓN
# ============================================================================

from dotenv import load_dotenv
load_dotenv()  # Esto carga variables desde un archivo .env si existe

# Configuración para Azure OpenAI usando variables de entorno (posiblemente del .env)
USE_AZURE = os.environ.get("USE_AZURE_OPENAI", "false").lower() == "true"
AZURE_API_BASE = os.environ.get("AZURE_API_BASE", "https://development-cursor-models.openai.azure.com/")
AZURE_API_KEY = os.environ.get("AZURE_API_KEY", "")
AZURE_API_VERSION = os.environ.get("AZURE_API_VERSION", "2024-02-15-preview")

USE_AZURE: True
AZURE_API_BASE: https://development-cursor-models.openai.azure.com/
AZURE_API_KEY: Set
AZURE_API_VERSION: 2024-12-01-preview


In [4]:
# Modelos (si usas Azure, estos son los nombres de los deployments)
RUBRIC_GENERATION_MODEL = os.environ.get("RUBRIC_GENERATION_MODEL", "gpt-4o-mini")
RUBRIC_JUDGE_MODEL = os.environ.get("RUBRIC_JUDGE_MODEL", "gpt-4o-mini")

# Inicializar cliente OpenAI o Azure OpenAI
if USE_AZURE:
    if not AZURE_API_KEY:
        print("⚠️  Advertencia: USE_AZURE_OPENAI=true pero AZURE_API_KEY no está configurada")
    client = AsyncAzureOpenAI(
        api_version=AZURE_API_VERSION,
        azure_endpoint=AZURE_API_BASE.rstrip('/'),
        api_key=AZURE_API_KEY or os.environ.get("OPENAI_API_KEY", "")
    )
    print(f"✓ Usando Azure OpenAI")
    print(f"  Endpoint: {AZURE_API_BASE}")
    print(f"  API Version: {AZURE_API_VERSION}")
    print(f"  Modelo generación: {RUBRIC_GENERATION_MODEL}")
    print(f"  Modelo juez: {RUBRIC_JUDGE_MODEL}")
else:
    client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    print(f"✓ Usando OpenAI estándar")

✓ Usando Azure OpenAI
  Endpoint: https://development-cursor-models.openai.azure.com/
  API Version: 2024-12-01-preview
  Modelo generación: gpt-5-mini
  Modelo juez: gpt-5-mini


In [24]:
# ============================================================================
# FUNCIONES AUXILIARES (replicadas del proyecto)
# ============================================================================

def extract_json_from_response(response: str) -> Optional[Dict]:
    """Extrae un objeto JSON de una respuesta de texto."""
    # Buscar JSON en bloques de código
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', response, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group(1))
        except json.JSONDecodeError:
            pass
    
    # Buscar JSON directo
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group(0))
        except json.JSONDecodeError:
            pass
    
    return None


async def llamar_llm(prompt: str, system_prompt: Optional[str] = None, model: str = "gpt-4o-mini") -> str:
    """Llama a un LLM de forma asíncrona (soporta OpenAI y Azure OpenAI)."""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    # Configurar parámetros según si es Azure o OpenAI estándar
    if USE_AZURE:
        # Para Azure OpenAI, usamos AsyncAzureOpenAI con la configuración correcta
        azure_client = AsyncAzureOpenAI(
            api_version=AZURE_API_VERSION,
            azure_endpoint=AZURE_API_BASE.rstrip('/'),
            api_key=AZURE_API_KEY or os.environ.get("OPENAI_API_KEY", "")
        )
        
        # En Azure, pasamos el modelo (deployment name) en el create()
        # Azure OpenAI usa max_completion_tokens en lugar de max_tokens
        # Este modelo no acepta temperature=0, solo el valor por defecto (1)
        response = await azure_client.chat.completions.create(
            model=model,
            messages=messages,
            max_completion_tokens=4000
        )
    else:
        # OpenAI estándar
        response = await client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
            max_tokens=4000
        )
    
    return response.choices[0].message.content

### Dataset FrontierScience

In [ ]:
# Cell 1: imports y carga del dataset FrontierScience-Research

import pandas as pd

df = pd.read_json("frontierscience-research/test.jsonl", lines=True)   # clave: lines=True
print(df.info())
print(df.head(10))

row_n = 2
row = df.loc[row_n]
question = df.loc[row_n, 'problem']
print(question)

### Dataset Chem

In [ ]:
# # TOY EXAMPLE

# # ========================================================================
# # PASO 1: Dataset Original
# # ========================================================================
# print("\n📋 PASO 1: Dataset Original")
# print("-" * 70)

# pregunta = "¿Qué es la inteligencia artificial?"


📋 PASO 1: Dataset Original
----------------------------------------------------------------------
Pregunta: ¿Qué es la inteligencia artificial?

Rubricas Persistentes: 2
  1. [Definición Clara] La respuesta debe definir claramente qué es la inteligencia artificial...
  2. [Ejemplos Prácticos] La respuesta debe incluir ejemplos prácticos de aplicaciones de IA...


In [ ]:
import pandas as pd

df = pd.read_csv('data/moose-chem/chem_research_2024-sample.csv')
df.head()

,No,Title,Public Date,Publisher,Background Little Survey,Background Little Survey (strict),Background Question,Background Question (strict),Main Inspiration,Inspiration paper 1 title,Relation between the main inspiration and the inspiration paper 1,Inspiration paper 2 title,Relation between the main inspiration and the inspiration paper 2,Inspiration paper 3 title,Relation between the main inspiration and the inspiration paper 3,Main hypothesis,Experiments to Verify the Research Hypothesis:,Reasoning Process,Note
0,0,"Ultrastrong, flexible thermogalvanic armor wit...",2024/08/07,Nature Communication,Background Survey Summary\nPrior to the develo...,Background Survey Summary\nPrior to the develo...,How can we design a flexible thermogalvanic de...,NaN,The main inspiration for the paper stems from ...,Interactions between macromolecules and ions: ...,Their work on the Hofmeister series (cited as ...,Beyond Hofmeister,Insights into using the Hofmeister series to m...,Strong tough hydrogels via the synergy of free...,This paper demonstrates the synergy of freeze-...,By integrating guanidine sulfate (Gdm)2SO4 int...,Thermoelectric Performance Tests: The FTGA exh...,bkg + insp1/insp2 + insp3 = hyp,insp1/2: The Hofmeister series; insp3: directi...
1,1,Chaotropic Effect-Boosted Thermogalvanic Ionog...,2024/01/29,Advanced Materials,Thermoelectric Energy Conversion: Traditional ...,Thermoelectric Energy Conversion: Traditional ...,How can a thermoelectric system be designed to...,How can a thermoelectric system be designed to...,The main inspiration of the paper comes from t...,The Chaotropic Effect as an Assembly Motif in ...,"Have the keyword ""Chaotropic Effect""",Role of Ions in Hydrogels with an Ionic Seebec...,This paper explains how the Soret effect and s...,Hierarchically porous polymer coatings for hig...,This paper details the creation of radiative c...,The hypothesis of the paper is that integratin...,The experiments included the design and testin...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Chaotropic Effect (ref id: find online ...
2,2,Synergistic Anisotropic Network and Hierarchic...,2024/02/01,Small,Background survey:\n\nHarnessing Low-Grade Hea...,NaN,How can a cost-effective N-type quasi-solid-st...,NaN,The main inspiration of this work is the syner...,3D Hierarchical Electrodes Boosting Ultrahigh ...,It directly inspires the design of the hierarc...,Cu(ii/i) redox couples: potential alternatives...,This work supports the use of copper as the re...,Anti-Fatigue and Highly Conductive Thermocells...,It directly inspired the structural design of ...,The hypothesis is that combining an anisotropi...,The experiments designed to test the hypothesi...,bkg + insp1 + insp2 + insp3 = hyp,insp1: Hierarchical 3D Copper Electrodes (find...
3,3,Liquid-flow thermocells with high hybrid entro...,2024/07/14,Nano Energy,Background Survey\nThermoelectric Conversion f...,NaN,How can the thermoelectric performance of liqu...,How can the thermoelectric performance of liqu...,The main inspiration of this paper is the hybr...,Effect of solvation shell structure on thermop...,It demonstrated theThis paper serves as a foun...,High thermopower of ferri/ferrocyanide redox c...,This paper introduces the concept of using a c...,NaN,NaN,The hypothesis is that inducing hybrid entropy...,The experiments include fabricating LFTCs with...,bkg + insp1 + insp2 = hyp,insp1: Hybrid Entropy Increase through Flow-Se...
4,4,Tough and elastic hydrogel thermocells for hea...,2024/08/01,Chemical Engineering Journal,Background Survey:\nHydrogel thermocells have ...,NaN,How can a hydrogel thermocell be designed to a...,How can a hydrogel thermocell be designed to a...,The main inspiration behind this paper is a bi...,"Fracture, fatigue, and friction of polymers in...",This paper focuses on the role of dense polyme...,Highly stretchable and tough hydrogels,"This paper shows how a dual-network approach, ...",A review of radiation-grafted polymer electrol...,increased m

In [ ]:
row_n = 0
row = df.loc[row_n]
background_survey = df.loc[row_n, 'Background Little Survey']
background_question = df.loc[row_n, 'Background Question']

print(f"Row: {row_n}")
print(f"Title: {df.loc[row_n, 'Title']}")
print(f"Public Date: {df.loc[row_n, 'Public Date']}")
print(f"Publisher: {df.loc[row_n, 'Publisher']}\n")
print(f"Background Little Survey: \n{df.loc[row_n, 'Background Little Survey']}\n")
print(f"Background Question: \n{df.loc[row_n, 'Background Question']}")

Row: 0
Title: Ultrastrong, flexible thermogalvanic armor with a Carnot-relative efficiency over 8%
Public Date: 2024/08/07
Publisher: Nature Communication

Background Little Survey: 
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.
2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid electrolytes, provided a 

In [17]:
question = f"""A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.

---
{background_survey}

Pregunta:
{background_question}

Tarea: Utilizando únicamente el resumen de contexto proporcionado, redacta una respuesta clara y completa a la pregunta indicada.
"""

print(question)

A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.

---
Background Survey Summary
Prior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:
Thermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.
2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid electrolytes, provided

### Generacion rubricas originales

In [ ]:
# # TOY EXAMPLE

# ground_truth_inicial = {
#     "query": pregunta,
#     "rubrics": [
#         {
#             "description": "La respuesta debe definir claramente qué es la inteligencia artificial",
#             "weight": 1.0,
#             "title": "Definición Clara"
#         },
#         {
#             "description": "La respuesta debe incluir ejemplos prácticos de aplicaciones de IA",
#             "weight": 1.0,
#             "title": "Ejemplos Prácticos"
#         }
#     ]
# }

# print(f"Pregunta: {pregunta}")
# print(f"\nRubricas Persistentes: {len(ground_truth_inicial['rubrics'])}")
# for i, rubric in enumerate(ground_truth_inicial["rubrics"], 1):
#     print(f"  {i}. [{rubric['title']}] {rubric['description']}...")

In [ ]:
async def generar_rubricas_originales(pregunta: str, model: str = None) -> Dict[str, Any]:
    if model is None:
        model = RUBRIC_GENERATION_MODEL
    
    prompt = f"""Eres un experto en evaluación educativa. Genera rubricas de evaluación para la siguiente pregunta.

Pregunta: {pregunta}

Genera 2-4 rubricas que cubran los aspectos esenciales para evaluar una respuesta a esta pregunta. 
Cada rubrica debe tener:
- title: Un título corto y descriptivo
- description: Una descripción detallada de qué se evalúa
- weight: Un peso (usa 1.0 para todas)

Responde SOLO con un JSON válido en este formato:
{{
  "query": "{pregunta}",
  "rubrics": [
    {{
      "title": "Título de la rubrica",
      "description": "Descripción detallada de qué se evalúa",
      "weight": 1.0
    }}
  ]
}}"""

    try:
        respuesta_llm = await llamar_llm(
            prompt=prompt,
            model=model
        )
        
        rubricas = extract_json_from_response(respuesta_llm)
        
        if rubricas and "rubrics" in rubricas:
            return rubricas
        else:
            print(f"⚠️  No se pudo extraer rubricas válidas")
            print(f"Respuesta recibida: {respuesta_llm[:300]}...")
            # Retornar estructura básica si falla
            return {
                "query": pregunta,
                "rubrics": [
                    {
                        "title": "Respuesta relevante",
                        "description": "La respuesta debe ser relevante a la pregunta",
                        "weight": 1.0
                    }
                ]
            }
    except Exception as e:
        print(f"❌ Error generando rubricas originales: {e}")
        return {
            "query": pregunta,
            "rubrics": [
                {
                    "title": "Respuesta relevante",
                    "description": "La respuesta debe ser relevante a la pregunta",
                    "weight": 1.0
                }
            ]
        }


In [ ]:
# Paso 2: Generar rubricas originales usando LLM
print("\n📋 PASO 1: Generando rubricas originales con LLM...")
print("-" * 70)

ground_truth_inicial = await generar_rubricas_originales(question)

print(f"✓ Rubricas generadas: {len(ground_truth_inicial['rubrics'])}")
for i, rubric in enumerate(ground_truth_inicial["rubrics"], 1):
    print(f"  {i}. [{rubric['title']}] {rubric['description'][:80]}...")



📋 PASO 1: Generando rubricas originales con LLM...
----------------------------------------------------------------------
✓ Rubricas generadas: 4
  1. [Adherencia al resumen] Evalúa si la respuesta se basa exclusivamente en la información proporcionada en...
  2. [Análisis del compromiso eficiencia–robustez] Mide la calidad del análisis sobre el trade-off entre eficiencia termogalvánica ...
  3. [Propuesta de diseño coherente con objetivos] Valora si la respuesta propone una solución de diseño que busque simultáneamente...
  4. [Claridad y lógica de la argumentación] Evalúa la claridad expositiva, la organización y la lógica interna de la respues...


In [29]:
ground_truth_inicial

{'query': "A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.\n\n---\nBackground Survey Summary\nPrior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:\nThermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.\n2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid elect

### Respuestas del modelo

In [ ]:
# # TOY EXAMPLE

# # ========================================================================
# # PASO 2: Respuestas del modelo
# # ========================================================================
# print("\n\n🤖 PASO 2: Respuestas generadas por el modelo")
# print("-" * 70)

# respuestas = [
#     "La inteligencia artificial (IA) es la capacidad de las máquinas de realizar tareas que normalmente requieren inteligencia humana, como el reconocimiento de patrones y el aprendizaje. Ejemplos incluyen asistentes virtuales como Siri y sistemas de recomendación.",
#     "La IA es cuando las computadoras hacen cosas inteligentes. Hay muchos ejemplos como robots y chatbots.",
#     "La inteligencia artificial es un campo de la informática que busca crear sistemas capaces de simular funciones cognitivas humanas. Incluye técnicas como machine learning, procesamiento de lenguaje natural y visión por computadora. Aplicaciones prácticas incluyen diagnóstico médico asistido por IA, traducción automática, vehículos autónomos, y sistemas de detección de fraudes financieros. La IA puede anticipar casos límite y manejar situaciones complejas.",
#     "La inteligencia artificial es cuando las máquinas aprenden de datos. Por ejemplo, cuando Netflix te recomienda películas basándose en lo que viste antes, eso es IA. También cuando los coches se conducen solos. La IA causa que las máquinas sean inteligentes porque hay más datos disponibles."
# ]

# print(f"Se generaron {len(respuestas)} respuestas")



🤖 PASO 2: Respuestas generadas por el modelo
----------------------------------------------------------------------
Se generaron 4 respuestas


In [ ]:
async def generar_respuestas_modelo(pregunta: str, num_respuestas: int = 4, model: str = None) -> List[str]:
    if model is None:
        model = RUBRIC_GENERATION_MODEL
    
    respuestas = []
    
    # Generar respuestas con diferentes instrucciones para obtener variedad
    instrucciones_variadas = [
        "Responde de manera completa y detallada, incluyendo ejemplos específicos.",
        "Responde de manera concisa pero informativa.",
        "Responde de manera muy detallada y técnica, incluyendo conceptos avanzados.",
        "Responde de manera simple y directa, adecuada para principiantes.",
        "Responde de manera equilibrada, balanceando profundidad y claridad.",
    ]
    
    for i in range(num_respuestas):
        instruccion = instrucciones_variadas[i % len(instrucciones_variadas)]
        
        prompt = f"""{instruccion}

Pregunta: {pregunta}

Responde la pregunta de manera clara y completa."""

        try:
            respuesta = await llamar_llm(
                prompt=prompt,
                model=model
            )
            respuestas.append(respuesta.strip())
        except Exception as e:
            print(f"⚠️  Error generando respuesta {i+1}: {e}")
            respuestas.append(f"Error al generar respuesta {i+1}")
    
    return respuestas


In [ ]:
# Paso 3: Generar respuestas del modelo usando LLM
print("\n\n🤖 PASO 2: Generando respuestas del modelo con LLM...")
print("-" * 70)

num_respuestas_auto = 4
answers = await generar_respuestas_modelo(question, num_respuestas=num_respuestas_auto)

print(f"✓ Respuestas generadas: {len(answers)}")
for i, respuesta in enumerate(answers, 1):
    print(f"\n  Respuesta {i} ({len(respuesta)} caracteres):")
    print(f"  {respuesta[:150]}...")




🤖 PASO 2: Generando respuestas del modelo con LLM...
----------------------------------------------------------------------
✓ Respuestas generadas: 4

  Respuesta 1 (4389 caracteres):
  Respuesta clara y completa (usando únicamente la información del resumen):

Para maximizar simultáneamente la eficiencia relativa a Carnot y la robust...

  Respuesta 2 (1636 caracteres):
  Respuesta resumida y concreta:

Para maximizar simultáneamente la eficiencia relativa a Carnot y la robustez mecánica en un dispositivo termogalvánico...

  Respuesta 3 (7976 caracteres):
  Respuesta clara y técnica (basada únicamente en el resumen proporcionado)

Objetivo de diseño
- Maximizar simultáneamente la eficiencia relativa a Car...

  Respuesta 4 (1758 caracteres):
  Respuesta simple y directa:

Diseña el dispositivo como una célula termoeléctrica quasi-sólida (gel que confina el electrolito) y aplica simultáneamen...


In [35]:
print(respuestas_auto[2])

Respuesta clara y técnica (basada únicamente en el resumen proporcionado)

Objetivo de diseño
- Maximizar simultáneamente la eficiencia relativa a Carnot (es decir, obtener la mayor fracción posible de la energía térmica utilizable desde el gradiente térmico corporal) y la robustez mecánica (flexibilidad, resistencia al cizallamiento, durabilidad) de un dispositivo termogalvánico flexible, partiendo de las limitaciones y avances descritos en el resumen.

Principios físicos y de compromiso relevantes (extraídos del resumen)
- La eficiencia termogalvánica relativa a Carnot se incrementa al aumentar la diferencia de entropía de los pares redox (mayor termopolaridad/termopoder), al mantener alta conductividad iónica y cinética redox, y al reducir pérdidas térmicas y resistivas internas.
- Los dispositivos quasi‑sólidos (geles que confinan electrólitos) permiten flexibilidad pero presentan un trade‑off entre robustez mecánica y rendimiento termoelectroquímico: reforzar la matriz suele degra

### Generar rubricas adaptativas

In [ ]:
# ============================================================================
# FUNCIÓN PRINCIPAL: GENERAR RUBRICAS ADAPTATIVAS
# ============================================================================

from evolving_rubrics.prompts import get_adaptive_rubrics_prompt

async def generar_rubricas_adaptativas(
    pregunta: str,
    respuestas: List[str],
    rubricas_existentes: Optional[List[Dict[str, Any]]] = None) -> Optional[Dict[str, Any]]:
    # Construir el prompt usando la función del módulo prompts
    prompt_completo = get_adaptive_rubrics_prompt(pregunta, respuestas, rubricas_existentes)
    
    try:
        # Llamar al LLM
        respuesta_llm = await llamar_llm(
            prompt=prompt_completo,
            model=RUBRIC_GENERATION_MODEL
        )
        
        # Extraer JSON
        rubricas = extract_json_from_response(respuesta_llm)
        
        if rubricas:
            return rubricas
        else:
            print(f"⚠️  No se pudo extraer JSON de la respuesta del LLM")
            print(f"Respuesta recibida: {respuesta_llm[:200]}...")
            return None
            
    except Exception as e:
        print(f"❌ Error generando rubricas: {e}")
        return None

In [38]:
# ========================================================================
# PASO 3: Generar rubricas adaptativas
# ========================================================================
print("\n\n🧠 PASO 3: Generar rubricas adaptativas")
print("-" * 70)
print("Analizando diferencias entre respuestas...")

rubricas_adaptativas = await generar_rubricas_adaptativas(
    pregunta=question,
    respuestas=answers,
    rubricas_existentes=ground_truth_inicial["rubrics"]
)

if rubricas_adaptativas:
    print("\n✓ Rubricas adaptativas generadas:")
    
    if rubricas_adaptativas.get("positive_rubrics"):
        print(f"\n  Rubricas POSITIVAS ({len(rubricas_adaptativas['positive_rubrics'])}):")
        for i, rubric in enumerate(rubricas_adaptativas["positive_rubrics"], 1):
            print(f"    {i}. [{rubric.get('title', 'Sin título')}]")
            print(f"       {rubric['description'][:80]}...")
    
    if rubricas_adaptativas.get("negative_rubrics"):
        print(f"\n  Rubricas NEGATIVAS ({len(rubricas_adaptativas['negative_rubrics'])}):")
        for i, rubric in enumerate(rubricas_adaptativas["negative_rubrics"], 1):
            print(f"    {i}. [{rubric.get('title', 'Sin título')}]")
            print(f"       {rubric['description'][:80]}...")
else:
    print("❌ No se pudieron generar rubricas adaptativas")




🧠 PASO 3: Generar rubricas adaptativas
----------------------------------------------------------------------
Analizando diferencias entre respuestas...

✓ Rubricas adaptativas generadas:

  Rubricas POSITIVAS (2):
    1. [Mapeo función→diseño (mecanismo explícito)]
       Expone un mapeo explícito y verificable entre cada elección de diseño y la métri...
    2. [Plan de optimización cuantificable y validación]
       Propone un plan de optimización y validación accionable con parámetros clarament...

  Rubricas NEGATIVAS (1):
    1. [Afirmaciones causales contradictorias o no respaldadas]
       Afirmaciones causales activas o conclusiones que contradicen o no están respalda...


In [ ]:
rubricas_adaptativas['positive_rubrics']

[{'description': 'Expone un mapeo explícito y verificable entre cada elección de diseño y la métrica física que mejora: por ejemplo, identifica qué elemento (GdmCl, doble red, canales anisotrópicos, geometría de capas) afecta concretamente la termopower/Carnot-relative efficiency, cuál afecta la movilidad iónica/resistencia interna y cuál aporta rigidez/tenacidad mecánica. Una respuesta excelente no se limita a enumerar técnicas; indica para cada técnica el mecanismo físico (p. ej. aumento de diferencia de entropía, efecto Hofmeister, reducción de tortuosidad) y cómo mitiga el compromiso eficiencia–robustez descrito en el resumen.',
  'title': 'Mapeo función→diseño (mecanismo explícito)'},
 {'description': 'Propone un plan de optimización y validación accionable con parámetros claramente afinables y métricas medibles: especifica variables a ajustar (por ejemplo, concentración de GdmCl, grado de reticulación de cada red, orientación/tortuosidad de canales, espesor de capas), las magnitu

In [41]:
rubricas_adaptativas['negative_rubrics']

[{'description': 'Afirmaciones causales activas o conclusiones que contradicen o no están respaldadas por el resumen: por ejemplo, afirmar que un aditivo como GdmCl fortalece la red polimérica (cuando el resumen dice lo contrario), o asegurar la eliminación del trade‑off sin explicar mecanismos plausibles citados en el resumen. Se penaliza cualquier afirmación categórica sobre efectos materiales o resultados (p. ej. aumento de densidad de potencia) que vaya en contra de las limitaciones y compromisos explicitados en el resumen.',
  'title': 'Afirmaciones causales contradictorias o no respaldadas'}]

### Actualizar GT

In [ ]:
# ============================================================================
# FUNCIÓN: ACTUALIZAR GROUND TRUTH
# ============================================================================

def actualizar_ground_truth(
    ground_truth_inicial: Dict[str, Any],
    rubricas_adaptativas: Dict[str, Any]
) -> Dict[str, Any]:
    ground_truth_actualizado = ground_truth_inicial.copy()
    
    # Obtener rubricas persistentes originales
    rubricas_persistentes = ground_truth_inicial.get("rubrics", [])
    
    # Convertir rubricas adaptativas al formato correcto
    rubricas_nuevas = []
    
    # Rubricas positivas (peso +1.0)
    for rubric in rubricas_adaptativas.get("positive_rubrics", []):
        rubricas_nuevas.append({
            "description": rubric["description"],
            "weight": 1.0,
            "title": rubric.get("title", "Sin título")
        })
    
    # Rubricas negativas (peso -1.0)
    for rubric in rubricas_adaptativas.get("negative_rubrics", []):
        rubricas_nuevas.append({
            "description": rubric["description"],
            "weight": -1.0,
            "title": rubric.get("title", "Sin título")
        })
    
    # Combinar: persistentes primero, luego adaptativas
    ground_truth_actualizado["rubrics"] = rubricas_persistentes + rubricas_nuevas
    
    # Agregar tipos (opcional, para tracking)
    tipos = ["persistent"] * len(rubricas_persistentes) + ["adaptive"] * len(rubricas_nuevas)
    ground_truth_actualizado["rubrics_types"] = tipos
    
    return ground_truth_actualizado


In [43]:
# ========================================================================
# PASO 4: Actualizar Ground Truth
# ========================================================================
print("\n\n📝 PASO 4: Actualizar Ground Truth")
print("-" * 70)

ground_truth_final = actualizar_ground_truth(
    ground_truth_inicial,
    rubricas_adaptativas
)

print(f"Rubricas totales: {len(ground_truth_final['rubrics'])}")
print(f"  - Persistentes: {len(ground_truth_inicial['rubrics'])}")
print(f"  - Adaptativas: {len(ground_truth_final['rubrics']) - len(ground_truth_inicial['rubrics'])}")



📝 PASO 4: Actualizar Ground Truth
----------------------------------------------------------------------
Rubricas totales: 7
  - Persistentes: 4
  - Adaptativas: 3


In [44]:
ground_truth_final

{'query': "A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.\n\n---\nBackground Survey Summary\nPrior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:\nThermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.\n2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel networks to confine liquid elect

### Evaluar respuestas

In [45]:
# ============================================================================
# FUNCIÓN: EVALUAR RESPUESTA CON UNA RUBRICA
# ============================================================================

async def evaluar_rubrica(
    respuesta: str,
    pregunta: str,
    descripcion_rubrica: str
) -> float:
    """
    Evalúa una respuesta contra un criterio específico usando un LLM juez.
    
    Args:
        respuesta: La respuesta a evaluar
        pregunta: La pregunta original
        descripcion_rubrica: Descripción del criterio a evaluar
    
    Returns:
        Score entre 0.0 y 1.0
    """
    system_prompt = """You will be given a question someone asked (in <question></question> tags) and the corresponding response (in <response></response> tags) given to them by an assistant. You will then be given a specific criterion of the response to evaluate (in <criterion></criterion> tags).
Return a score on a scale of 0 to 2 indicating how appropriate the response is based on the given criterion. Judge only the specified aspect(s), not any other qualities of the answer. Output JSON in the format: {"score": x}."""
    
    user_prompt = f"""<question>{pregunta}</question>
<response>{respuesta}</response>
<criterion>{descripcion_rubrica}</criterion>"""
    
    try:
        respuesta_llm = await llamar_llm(
            prompt=user_prompt,
            system_prompt=system_prompt,
            model=RUBRIC_JUDGE_MODEL
        )
        
        resultado = extract_json_from_response(respuesta_llm)
        
        if resultado and "score" in resultado:
            score = float(resultado["score"])
            # Normalizar de 0-2 a 0-1
            return score / 2.0
        else:
            return 0.0
            
    except Exception as e:
        print(f"⚠️  Error evaluando rubrica: {e}")
        return 0.0


# ============================================================================
# FUNCIÓN: EVALUAR RESPUESTA CON TODAS LAS RUBRICAS
# ============================================================================

async def evaluar_respuesta_completa(
    respuesta: str,
    ground_truth: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Evalúa una respuesta contra todas las rubricas y calcula recompensa total.
    
    Args:
        respuesta: La respuesta a evaluar
        ground_truth: Ground truth con todas las rubricas
    
    Returns:
        Diccionario con scores por rubrica y recompensa total
    """
    pregunta = ground_truth["query"]
    rubricas = ground_truth["rubrics"]
    
    # Evaluar cada rubrica
    scores_por_rubrica = {}
    tareas = []
    
    for rubric in rubricas:
        descripcion = rubric["description"]
        # Crear clave única para la rubrica
        clave = rubric.get("title", descripcion[:30])
        tareas.append((clave, evaluar_rubrica(respuesta, pregunta, descripcion)))
    
    # Ejecutar todas las evaluaciones en paralelo
    resultados = await asyncio.gather(*[tarea[1] for tarea in tareas])
    
    for (clave, _), score in zip(tareas, resultados):
        scores_por_rubrica[clave] = score
    
    # Calcular recompensa total (promedio ponderado)
    recompensa_total = 0.0
    peso_total = 0.0
    
    for i, rubric in enumerate(rubricas):
        clave = rubric.get("title", rubric["description"][:30])
        peso = abs(rubric["weight"])  # Usar valor absoluto para el peso
        score = scores_por_rubrica[clave]
        
        # Multiplicar por el signo del weight (positivo o negativo)
        recompensa_total += score * rubric["weight"] * peso
        peso_total += peso
    
    recompensa_final = recompensa_total / peso_total if peso_total > 0 else 0.0
    
    return {
        "recompensa_total": recompensa_final,
        "scores_por_rubrica": scores_por_rubrica,
        "num_rubricas": len(rubricas)
    }

In [ ]:
# ========================================================================
# PASO 5: Evaluar respuestas
# ========================================================================
print("\n\n📊 PASO 5: Evaluar respuestas con todas las rubricas")
print("-" * 70)

evaluaciones = []
for i, respuesta in enumerate(answers, 1):
    print(f"\n  Evaluando Respuesta {i}...")
    resultado = await evaluar_respuesta_completa(respuesta, ground_truth_final)
    evaluaciones.append({
        'respuesta_num': i,
        **resultado
    })
    print(f"    Recompensa total: {resultado['recompensa_total']:.3f}")



📊 PASO 5: Evaluar respuestas con todas las rubricas
----------------------------------------------------------------------

  Evaluando Respuesta 1...
    Recompensa total: 0.643

  Evaluando Respuesta 2...
    Recompensa total: 0.643

  Evaluando Respuesta 3...
    Recompensa total: 0.643

  Evaluando Respuesta 4...
    Recompensa total: 0.714


### Resumen

In [48]:
# ========================================================================
# RESUMEN
# ========================================================================
print("\n\n" + "="*70)
print("📈 RESUMEN FINAL")
print("="*70)

print(f"\nRubricas:")
print(f"  - Persistentes: {len(ground_truth_inicial['rubrics'])}")
print(f"  - Adaptativas: {len(ground_truth_final['rubrics']) - len(ground_truth_inicial['rubrics'])}")
print(f"  - Total: {len(ground_truth_final['rubrics'])}")

print(f"\nRecompensas por respuesta:")
for eval_result in evaluaciones:
    print(f"  Respuesta {eval_result['respuesta_num']}: {eval_result['recompensa_total']:.3f}")

mejor = max(evaluaciones, key=lambda x: x['recompensa_total'])
peor = min(evaluaciones, key=lambda x: x['recompensa_total'])

print(f"\n✓ Mejor respuesta: Respuesta {mejor['respuesta_num']} (recompensa: {mejor['recompensa_total']:.3f})")
print(f"✗ Peor respuesta: Respuesta {peor['respuesta_num']} (recompensa: {peor['recompensa_total']:.3f})")

print("\n" + "="*70)
print("✓ Proceso completado!")
print("="*70)

output = {
    'ground_truth_inicial': ground_truth_inicial,
    'rubricas_adaptativas': rubricas_adaptativas,
    'ground_truth_final': ground_truth_final,
    'evaluaciones': evaluaciones
    }



📈 RESUMEN FINAL

Rubricas:
  - Persistentes: 4
  - Adaptativas: 3
  - Total: 7

Recompensas por respuesta:
  Respuesta 1: 0.643
  Respuesta 2: 0.643
  Respuesta 3: 0.643
  Respuesta 4: 0.714

✓ Mejor respuesta: Respuesta 4 (recompensa: 0.714)
✗ Peor respuesta: Respuesta 1 (recompensa: 0.643)

✓ Proceso completado!


In [49]:
output

{'ground_truth_inicial': {'query': "A continuación se muestra un resumen de contexto de un trabajo científico, seguido de una pregunta que debe responderse usando únicamente la información contenida en dicho resumen.\n\n---\nBackground Survey Summary\nPrior to the development of the flexible thermogalvanic device described in this paper, there were significant advances in thermoelectric and thermogalvanic technologies, particularly in the context of energy harvesting from body heat. Key technologies included:\nThermoelectric Devices: Traditional semiconductor thermoelectric generators (TEGs), while effective at converting heat to electricity, often faced limitations in efficiency, especially when dealing with low-grade heat sources like body heat. These devices typically suffered from low thermopower and were limited by rigid structures, which made them less suitable for wearable applications.\n2. Quasi-Solid Thermocells: The emergence of quasi-solid thermocells, which used gel network

In [51]:
evaluaciones

[{'respuesta_num': 1,
  'recompensa_total': 0.6428571428571429,
  'scores_por_rubrica': {'Adherencia al resumen': 1.0,
   'Análisis del compromiso eficiencia–robustez': 1.0,
   'Propuesta de diseño coherente con objetivos': 1.0,
   'Claridad y lógica de la argumentación': 1.0,
   'Mapeo función→diseño (mecanismo explícito)': 1.0,
   'Plan de optimización cuantificable y validación': 0.5,
   'Afirmaciones causales contradictorias o no respaldadas': 1.0},
  'num_rubricas': 7},
 {'respuesta_num': 2,
  'recompensa_total': 0.6428571428571429,
  'scores_por_rubrica': {'Adherencia al resumen': 0.5,
   'Análisis del compromiso eficiencia–robustez': 1.0,
   'Propuesta de diseño coherente con objetivos': 1.0,
   'Claridad y lógica de la argumentación': 1.0,
   'Mapeo función→diseño (mecanismo explícito)': 1.0,
   'Plan de optimización cuantificable y validación': 0.5,
   'Afirmaciones causales contradictorias o no respaldadas': 0.5},
  'num_rubricas': 7},
 {'respuesta_num': 3,
  'recompensa_tota

# DEV GRubrics-Science (general)

### Cargar dataset

In [38]:
# Cell 1: imports y carga del dataset FrontierScience-Research

import pandas as pd
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any
import numpy as np
import math
import random

df = pd.read_json("frontierscience-research/test.jsonl", lines=True)   # clave: lines=True
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   problem        60 non-null     object
 1   answer         60 non-null     object
 2   subject        60 non-null     object
 3   task_group_id  60 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB
None


,problem,answer,subject,task_group_id
0,Context: Weak value amplification is an enhanc...,"Points: 1.5, Item: Background Theory - Qualita...",physics,222e24bf-14d4-4660-a5dd-01b2b3528cfa
1,"Context: For few-body system problems, a typic...","Points: 1.0, Item: Correctly identify that the...",physics,d3acd2cb-8477-4630-b8bb-b4bc22670dc8
2,Context: Weak value amplification (WVA) is an ...,"Points: 1.0, Item: Describes the time evoluti...",physics,bc82c611-4407-4da9-8444-48dbf08d0bbf
3,Context: The Mach-Zehnder Interferometer is a ...,"Points: 1.0, Item: Alternative Protocol Using ...",physics,1a5d24e0-9af4-4b36-98d1-2845df696d90
4,Context: This is a long and difficult problem ...,"Points: 1.0, Item: Derives the relation descri...",physics,790baca4-d2d8-4acb-8494-91adf77e026b
5,Context: The observation of neutrinos from SN1...,"Points: 1.0, Item: The solution to question 1 ...",physics,8fac71ae-1f4a-4edd-9f0d-88a443912a04
6,Context: Generalized (or weakened) quantum mea...,"Points: 0.5, Item: Defines the measurement ope...",physics,af50243e-3a60-4460-9536-f9a02c4f8eb8
7,Context: In the cosmic microwave background (C...,"Points: 1.0, Item: Mentions preprocessing trai...",physics,1c77dfab-266d-40d4-b3a0-2c27f0b32139
8,Context: Conventional unitary inversion can op...,"Points: 1.0, Item: Shows that the \\( \\ket{\\...",physics,07fd78cd-81de-4a62-9952-369093ec303f
9,Context: Deriving the effective Hamiltonian fo...,"Points: 1.0, Item: Correctly shows the Heisenb...",physics,d50f3722-14fb-485a-a417-ed6ae79f3c07


In [41]:
print(df.iloc[0].problem)

Context: Weak value amplification is an enhanced detection scheme that was first suggested by Aharonov, Albert, and Vaidman \[1\]. (See \[2\] and \[3\] for recent reviews.) The scheme exploits the fact that postselecting the weak measurement of an ancilla can produce a linear detector response with an anomalously high sensitivity to small changes in an interaction parameter. The sensitivity arises from coherent “super-oscillatory” interference in the ancilla, which is controlled by the choice of preparation and postselection of the ancilla. The price that one pays for this increase in sensitivity is a reduction in the potential signal (and thus the potential precision of any estimation) due to the postselection process. Nevertheless, by using this technique one can still consistently recover a large fraction of the maximum obtainable signal in a relatively simple way. The relevant information is effectively concentrated into the small set of rarely postselected events.

Some recent exp

In [40]:
print(df.iloc[0].answer)

Points: 1.5, Item: Background Theory - Qualitatively describes the procedure of Weak Value Amplification. Assign full points if as follows:


- **(0.5pts)** States that the meter and ancilla are then weakly coupled using an interaction Hamiltonian \\( H \\)
- **(0.5pts)** Mentions that the ancilla is postselected into some pure final state \\(|\\Psi_f\\rangle\\)
- **(0.5pts)** States that this procedure effectively prepares an enhanced meter state that includes the effect of the ancilla \\(|\\phi'\\rangle = \\hat{M}|\\phi\\rangle / |\\hat{M}|\\phi\\rangle|\\), which we write here in terms of a Kraus operator \\(\\hat{M} = \\langle \\Psi_f | \\exp(-ig \\hat{A} \\otimes \\hat{F}) |\\Psi_i\\rangle\\).
Points: 0.5, Item: Background Theory - Recalls that for a typical weak value amplification experiment one uses an interaction Hamiltonian of the form:

\\\[ \\hat{H}\_{\\text{int}} = h g \\hat{A} \\otimes \\hat{F} \\delta(t - t_0), \\\]

where \\(\\hat{A}\\) is an ancilla observable, \\(\\ha

## Ejecutar

## Analizar resultados